In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import warnings
from pathlib import Path
from config.constants import FINANCE_FEE_PATH
from r2r_pipelines.utils import assign_intake_cycle, create_pg_connection

warnings.filterwarnings("ignore")

## International total fees dataset
def transform_fees_by_segment(file_path = FINANCE_FEE_PATH, 
                            file_name = "TU+TC Total Tuition Fees by Segment.xlsx", 
                            sheet_name = 'TU'):
    # Read the excel file
    df = pd.read_excel(Path(file_path)/file_name, sheet_name=sheet_name, header=5)

    # Renaming the first three columns and reformatting column names
    df.rename(columns={
        "Unnamed: 0": "prog_name",
        "Unnamed: 1": "intake",
        "Unnamed: 2": "intake_semester"
    }, inplace=True)
    
    df.columns = df.columns.str.lower().str.replace(r"[() ]", "_", regex=True)

    # Keep only rows with "semester" in the 'intake_semester' column
    df = df[df['intake_semester'].str.contains("Semester", na=False, case=False)]

    # Remove the 'INACTIVE' string from the 'prog_name' column
    df['prog_name'] = df['prog_name'].str.replace(r'\(INACTIVE\)|- INACTIVE', '', regex=True).str.strip()

    # Reformat the 'intake_semester' column to only contain the month integer
    df['intake_semester'] = df['intake_semester'].str.extract(r'(\d+)').astype(int)
    
    return df

def extract_transform_fees_by_segment():
    df = pd.concat(
        [transform_fees_by_segment(sheet_name=sheet) for sheet in ['TU', 'TC']], ignore_index=True
        ).rename(columns={'total_tuition_fees__local_': 'total_tuition_fees_local',
                          'total_tuition_fee_per_student__international_': 'total_tuition_fees_international'})\
        .reset_index(drop=True)
    
    relevant_cols = ['prog_name', 'intake', 'intake_semester', 
                     'total_tuition_fees_local', 'total_tuition_fees_international']
    df = df[relevant_cols]
    
    return df

## Academic calendar dataset
def transform_acad_calendar(file_path = FINANCE_FEE_PATH,
                             file_name = "TUSB and TMSB - TM1 Acad Calendar.xlsx",
                             sheet_name = 'TUSB'):
    df = pd.read_excel(Path(file_path)/file_name, sheet_name=sheet_name, header=5)

    # Renaming the first three columns and reformatting column names
    df.rename(columns={
        "Unnamed: 0": "prog_name",
        "Unnamed: 1": "intake",
        "Unnamed: 2": "semester"
    }, inplace=True)

    df.columns = df.columns.str.lower().str.replace(r"[() ]", "_", regex=True)
    
    # Extract the digit from intake_semester if it contains "Semester", else set as <NA>
    df['intake_semester'] = df['semester'].apply(
        lambda x: int(x.split()[-1]) if isinstance(x, str) and "Semester" in x else pd.NA
    ).astype('Int64')

    df['intake_semester'].fillna(df['semester'], inplace=True)

    # Remove the 'INACTIVE' string from the 'prog_name' column
    df['prog_name'] = df['prog_name'].str.replace(r'\(INACTIVE\)|- INACTIVE', '', regex=True).str.strip()

    return df

def extract_transform_acad_calendar():
    df = pd.concat(
        [transform_acad_calendar(sheet_name=sheet) for sheet in ['TUSB', 'TMSB']], ignore_index=True
        ).reset_index(drop=True)
    
    # Convert start_month and end_month to datetime format
    df['start_month'] = pd.to_datetime(df['start_month'], errors='coerce', format='%b-%y')
    df['end_month'] = pd.to_datetime(df['end_month'], errors='coerce', format='%b-%y')
        
    relevant_cols = ['prog_name', 'intake', 'intake_semester', 'start_month', 'end_month']
    df = df[relevant_cols]
        
    return df

## CALSACE table
def extract_transform_calsace(file_path = FINANCE_FEE_PATH, file_name = "BI_Extract_TMStudentPercent_TC.csv"):
    df = pd.read_csv(Path(file_path)/file_name)

    # Rename columns
    rename_dict = {
        "ProgrammeName": 'prog_name',
        "Intake": 'intake',
        'StudentType': 'student_type',
        '%CAL4Subjects': 'cal_4_subjects', 
        '%1ScienceSubject': 'perc_1_science_subject', 
        '%2ScienceSubject': 'perc_2_science_subject'
    }
    df.rename(columns=rename_dict, inplace=True)

    # Filter rows and select relevant columns
    df = df[df['4DigitsCode'].isin(['CALH', 'SAMH'])].iloc[:, 2:]
    
    # Remove the 'INACTIVE' string from the 'prog_name' column
    df['prog_name'] = df['prog_name'].str.replace(r'\(INACTIVE\)|- INACTIVE', '', regex=True).str.strip()
    
    # Calculate calsace fees multipliers
    df['calsace_fee_mult_loc'] = np.where(df['student_type'] == 'New - Local', df['cal_4_subjects'] * 1/3, 0)
    df['calsace_fee_mult_intl'] = np.where(df['student_type'] == 'New - International', df['cal_4_subjects'] * 1/3, 0)
    df['calsace_sci_fee_mult_loc'] = np.where(df['student_type'] == 'New - Local', df['perc_1_science_subject'] + 2 * df['perc_2_science_subject'], 0)
    df['calsace_sci_fee_mult_intl'] = np.where(df['student_type'] == 'New - International', df['perc_1_science_subject'] + 2 * df['perc_2_science_subject'], 0)
    
    # Finalize columns
    final_cols = ['prog_name', 'intake',
                  'calsace_fee_mult_loc', 'calsace_fee_mult_intl',
                  'calsace_sci_fee_mult_loc', 'calsace_sci_fee_mult_intl']

    df = df[final_cols]
    
    df = df.groupby(['prog_name', 'intake']).agg({
            'calsace_fee_mult_loc': 'max',
            'calsace_fee_mult_intl': 'max',
            'calsace_sci_fee_mult_loc': 'max',
            'calsace_sci_fee_mult_intl': 'max'
        }).reset_index()
    
    return df

def extract_fin_fees_pgsql():
    fin_fee_query = """SELECT * FROM r2r_finance_fees"""
    engine = create_pg_connection()
    
    with engine.connect() as connection:
        df = pd.read_sql_query(fin_fee_query, connection)
    print("Data loaded successfully from cms_sas database")
    return df

def extract_fin_fees_manual(file_path = FINANCE_FEE_PATH, file_name = "E_FinanceFee_manual.xlsx"):
    # Read the excel file
    return pd.read_excel(Path(file_path)/file_name, sheet_name="C_FinanceFee", header=0)

def extract_transform_fin_fees(source='pgsql'):
    if source == 'pgsql':
        # Extract data from PostgreSQL database
        fin_df = extract_fin_fees_pgsql()
    else:
        # Extract data from manual Excel file
        print("Using manual finance fees data extraction...")
        fin_df = extract_fin_fees_manual()
    
    # Renaming columns
    rename_dict = {
        'course_desc_tm1': "prog_name",
        "course_desc_jarvis": "prog_name_jarvis",
        "intake": "intake",
        'semester': 'intake_semester',
        "year": "intake_year",
        'int_enrollment_fee': 'intl_enrollment_fee',
        'int_student_charges': 'intl_student_charges',
        'int_annual_fee': 'intl_annual_fee',
        'tmsciencefee': 'calsace_science_fee'
    }
    fin_df.rename(columns=rename_dict, inplace=True)

    # Select relevant columns
    fin_relevant_cols = ['prog_name', 'prog_name_jarvis', 'intake', 'intake_semester', 'intake_year', 'campus', 
                         'start_date', 'end_date', 'attrition', 'cms_progcode',
                         'intl_enrollment_fee', 'intl_student_charges', 'intl_annual_fee', 'int_total_fee',
                         'loc_enrollment_fee', 'loc_resource_fee', 'loc_tuition_fee', 'calsace_science_fee']
    fin_df = fin_df[fin_relevant_cols]
    
    # Remove the 'INACTIVE' string from the 'prog_name' column
    fin_df['prog_name'] = fin_df['prog_name'].str.replace(r'\(INACTIVE\)|- INACTIVE', '', regex=True).str.strip()
    
    # Keep rows with intake_month 202001 onwards
    fin_df = fin_df[fin_df['intake'] > 201900].reset_index(drop=True)
    
    # Remove duplicates from the prog_name, intake_month, intake_semester columns
    fin_df = fin_df.drop_duplicates(subset=['prog_name', 'intake', 'intake_semester'])
    
    return fin_df

def preprocess_finance_fees():
    # load data
    print("Start preprocessing finance fee files...")
    print("Loading data...")
    total_fees_df = extract_transform_fees_by_segment()
    calsace_df = extract_transform_calsace()
    acadcalendar_df = extract_transform_acad_calendar()
    fin_df = extract_transform_fin_fees(source='manual')
    
    print("Merging 'international total fees' data...")
    # Merge fin_df and total_fees_df on prog_name, intake_month, and intake_semester
    fin_merged = fin_df.merge(total_fees_df, on=['prog_name', 'intake', 'intake_semester'], how='left')
    
    # Remove duplicates based on prog_name, intake_month, and intake_semester
    fin_merged.drop_duplicates(subset=['prog_name', 'intake', 'intake_semester'], inplace=True)

    # Calculate total tuition fee based on the total fees file (intl)
    fin_merged['loc_tuition_fee'] = fin_merged['total_tuition_fees_local'].fillna(fin_merged['loc_tuition_fee'])
    fin_merged['intl_tuition_fee'] = fin_merged['total_tuition_fees_international'].fillna(fin_merged['loc_tuition_fee'])

    print("Merging 'academic calendar' data...")
    # Merge with academic calendar to obtain the academic start and end dates
    fin_merged = fin_merged.merge(acadcalendar_df, on=['prog_name', 'intake', 'intake_semester'], how='left')

    fin_merged['acad_start_date'] = fin_merged['start_date'].fillna(fin_merged['start_month'])
    fin_merged['acad_end_date'] = fin_merged['end_date'].fillna(fin_merged['end_month'])

    # Convert acad_start_date and acad_end_date to datetime format
    fin_merged['acad_start_date'] = pd.to_datetime(fin_merged['acad_start_date'], errors='coerce')
    fin_merged['acad_end_date'] = pd.to_datetime(fin_merged['acad_end_date'], errors='coerce')

    print("Merging 'CALSACE' data...")
    # Merge with calsace data
    fin_merged = fin_merged.merge(calsace_df, on=['prog_name', 'intake'], how='left')

    # Split intake into intake_month and intake_year
    fin_merged['intake_month'] = fin_merged['intake'] % 100
    fin_merged['intake_year'] = fin_merged['intake'] // 100

    # Create intake_cycle column
    fin_merged['intake_cycle'] = assign_intake_cycle(fin_merged, 'intake_month')

    print("Calculating amortization related columns...")
    # Create amortized_nom column
    fin_merged['amortized_nom'] = np.where(
        fin_merged['acad_start_date'].dt.year != fin_merged['acad_end_date'].dt.year,
        (12 - fin_merged['acad_start_date'].dt.month) + 12 * (fin_merged['acad_end_date'].dt.year - fin_merged['acad_start_date'].dt.year - 1) + 1,
        (fin_merged['acad_end_date'].dt.month - fin_merged['acad_start_date'].dt.month) + 1
    )

    # Calculate the amortized_denom
    fin_merged['amortized_denom'] = (
        (fin_merged['acad_end_date'].dt.year - fin_merged['acad_start_date'].dt.year) * 12 +
        fin_merged['acad_end_date'].dt.month - fin_merged['acad_start_date'].dt.month + 1
    )

    # Finalize columns
    fin_cols = [
        'prog_name', 'intake', 'intake_semester', 'intake_month', 'intake_cycle', 'intake_year', 'campus',
        'acad_start_date', 'acad_end_date', 'attrition',
        'intl_enrollment_fee', 'intl_student_charges', 'intl_annual_fee', 'intl_tuition_fee',
        'loc_enrollment_fee', 'loc_resource_fee', 'loc_tuition_fee', 
        'calsace_science_fee', 'calsace_fee_mult_loc', 'calsace_fee_mult_intl',
        'calsace_sci_fee_mult_loc', 'calsace_sci_fee_mult_intl', 'amortized_nom', 'amortized_denom'
    ]
    print("Preprocessing finance fee files completed.")
    
    return fin_merged[fin_cols]

## Calculate first year fees

In [2]:
fin_fee = preprocess_finance_fees()

# Select only the rows where the academic start date year is equal to the intake year, as we are calculating only the first year fees
fee_by_cycle = fin_fee[(fin_fee['acad_start_date'].dt.year == fin_fee['intake_year'])].reset_index(drop=True)

# Amortization formula is only applicable from 2023 onwards
fee_by_cycle['amortized_nom'] = fee_by_cycle.apply(
    lambda row: row['amortized_nom'] if row['intake_year'] >= 2023 else row['amortized_denom'], axis=1
    )

Start preprocessing finance fee files...
Loading data...
Using manual finance fees data extraction...
Merging 'international total fees' data...
Merging 'academic calendar' data...
Merging 'CALSACE' data...
Calculating amortization related columns...
Preprocessing finance fee files completed.


In [3]:
fin_fee

,prog_name,intake,intake_semester,intake_month,intake_cycle,intake_year,campus,acad_start_date,acad_end_date,attrition,...,loc_enrollment_fee,loc_resource_fee,loc_tuition_fee,calsace_science_fee,calsace_fee_mult_loc,calsace_fee_mult_intl,calsace_sci_fee_mult_loc,calsace_sci_fee_mult_intl,amortized_nom,amortized_denom
0,TMSB - South Australia Matriculation / SACE In...,201901,1,1,C1,2019,TC,2019-01-04,2019-02-21,0.00,...,650,2830,9320.0,350,0.0,0.0,0.8,0.8,2.0,2.0
1,TMSB - South Australia Matriculation / SACE In...,201901,2,1,C1,2019,TC,2019-03-22,2019-06-27,0.05,...,0,2830,9320.0,350,0.0,0.0,0.8,0.8,4.0,4.0
2,TMSB_Cambridge A-Levels,201901,1,1,C1,2019,TC,2019-01-04,2019-02-21,0.00,...,650,2830,8850.0,350,NaN,NaN,NaN,NaN,2.0,2.0
3,TMSB_Cambridge A-Levels,201901,2,1,C1,2019,TC,2019-03-22,2019-06-27,0.05,...,0,2830,8850.0,350,NaN,NaN,NaN,NaN,4.0,4.0
4,TMSB_Cambridge A-Levels,201901,3,1,C1,2019,TC,2019-08-16,2019-11-21,0.05,...,0,2830,8850.0,350,NaN,NaN,NaN,NaN,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10868,Doctor of Veterinary Medicine,202509,10,9,C3,2025,TU,2028-09-01,2029-01-01,0.04,...,0,0,50520.0,0,NaN,NaN,NaN,NaN,4.0,5.0
10869,Doctor of Veterinary Medicine,202509,11,9,C3,2025,TU,2029-02-01,2029-03-01,0.04,...,0,0,5052.0,0,NaN,NaN,NaN,NaN,2.0,2.0
10870,Doctor of Veterinary Medicine,202509,12,9,C3,2025,TU,2029-04-01,2029-07-01,0.04,...,0,0,50520.0,0,NaN,NaN,NaN,NaN,4.0,4.0
10871,Doctor of Veterinary Medicine,202509,13,9,C3,2025,TU,2029-09-01,1930-01-01,0.04,...,0,0,42942.0,0,NaN,NaN,NaN,NaN,-1196.0,-1195.0


In [12]:
first_year_fee = fee_by_cycle.groupby(['prog_name', 'campus', 'intake_year', 'intake_cycle', 'intake']).apply(
        lambda df: pd.Series({
            'fee_period_start': df['acad_start_date'].min(),
            'fee_period_end': df['acad_end_date'].max(),
            # Local Fees raw
            'loc_non_tuition_fees_actual': (df['loc_enrollment_fee'] + df['loc_resource_fee'] + 
                                            df['calsace_sci_fee_mult_loc'].fillna(0) * df['calsace_science_fee']).sum(),
            'loc_tuition_fees_actual': (df['loc_tuition_fee'] + df['loc_tuition_fee'] * 
                                                df['calsace_fee_mult_loc'].fillna(0)).sum(),
            # Local fees adjusted for amortization
            'loc_non_tuition_fees_amortized': ((df['loc_enrollment_fee'] + df['loc_resource_fee'] + 
                                                df['calsace_sci_fee_mult_loc'].fillna(0) * df['calsace_science_fee']) * 
                                                df['amortized_nom'] / df['amortized_denom']).sum(),
            'loc_tuition_fees_amortized': ((df['loc_tuition_fee'] + df['loc_tuition_fee'] * 
                                    df['calsace_fee_mult_loc'].fillna(0)) * 
                                    df['amortized_nom'] / df['amortized_denom']).sum(),
            # Local fees adjusted for amortization and attrition
            'loc_non_tuition_fees_attrited': ((df['loc_enrollment_fee'] + df['loc_resource_fee'] + 
                                                df['calsace_sci_fee_mult_loc'].fillna(0) * df['calsace_science_fee']) * 
                                                df['amortized_nom'] / df['amortized_denom'] * (1-df['attrition'])).sum(),
            'loc_tuition_fees_attrited': ((df['loc_tuition_fee'] + df['loc_tuition_fee'] * 
                                    df['calsace_fee_mult_loc'].fillna(0)) * 
                                    df['amortized_nom'] / df['amortized_denom'] * (1-df['attrition'])).sum(),
            # International Fees raw
            'intl_non_tuition_fees_actual': (df['intl_enrollment_fee'] + df['loc_resource_fee'] + 
                                            df['intl_student_charges'] + df['intl_annual_fee'] + 
                                            df['calsace_sci_fee_mult_intl'].fillna(0) * df['calsace_science_fee']).sum(),
            'intl_tuition_fees_actual': (df['intl_tuition_fee'] + df['intl_tuition_fee'] * 
                                        df['calsace_fee_mult_intl'].fillna(0)).sum(),
            # International fees adjusted for amortization
            'intl_non_tuition_fees_amortized': ((df['intl_enrollment_fee'] + df['loc_resource_fee'] + 
                                        df['intl_student_charges'] + df['intl_annual_fee'] + 
                                        df['calsace_sci_fee_mult_intl'].fillna(0) * df['calsace_science_fee']) * 
                                        df['amortized_nom'] / df['amortized_denom']).sum(),
            'intl_tuition_fees_amortized': ((df['intl_tuition_fee'] + df['intl_tuition_fee'] * 
                                    df['calsace_fee_mult_intl'].fillna(0)) * 
                                    df['amortized_nom'] / df['amortized_denom']).sum(),
            # International fees adjusted for amortization and attrition
            'intl_non_tuition_fees_attrited': ((df['intl_enrollment_fee'] + df['loc_resource_fee'] + 
                                        df['intl_student_charges'] + df['intl_annual_fee'] + 
                                        df['calsace_sci_fee_mult_intl'].fillna(0) * df['calsace_science_fee']) * 
                                        df['amortized_nom'] / df['amortized_denom'] * (1-df['attrition'])).sum(),
            'intl_tuition_fees_attrited': ((df['intl_tuition_fee'] + df['intl_tuition_fee'] * 
                                    df['calsace_fee_mult_intl'].fillna(0)) * 
                                    df['amortized_nom'] / df['amortized_denom'] * (1-df['attrition'])).sum(),
            # amortization flags     
            'amortized_nom': df['amortized_nom'].sum(),
            'amortized_denom': df['amortized_denom'].sum(),
        })
    ).reset_index()

In [5]:
first_year_fee['loc_total_fees_actual'] = first_year_fee['loc_tuition_fees_actual'] + first_year_fee['loc_non_tuition_fees_actual']
first_year_fee['loc_total_fees_amortized'] = first_year_fee['loc_tuition_fees_amortized'] + first_year_fee['loc_non_tuition_fees_amortized']
first_year_fee['loc_total_fees_attrited'] = first_year_fee['loc_tuition_fees_attrited'] + first_year_fee['loc_non_tuition_fees_attrited']

first_year_fee['intl_total_fees_actual'] = first_year_fee['intl_tuition_fees_actual'] + first_year_fee['intl_non_tuition_fees_actual']
first_year_fee['intl_total_fees_amortized'] = first_year_fee['intl_tuition_fees_amortized'] + first_year_fee['intl_non_tuition_fees_amortized']
first_year_fee['intl_total_fees_attrited'] = first_year_fee['intl_tuition_fees_attrited'] + first_year_fee['intl_non_tuition_fees_attrited']

In [6]:
first_year_fee

,prog_name,campus,intake_year,intake_cycle,intake,fee_period_start,fee_period_end,loc_non_tuition_fees_actual,loc_tuition_fees_actual,loc_non_tuition_fees_amortized,...,intl_non_tuition_fees_amortized,intl_tuition_fees_amortized,intl_non_tuition_fees_attrited,intl_tuition_fees_attrited,loc_total_fees_actual,loc_total_fees_amortized,loc_total_fees_attrited,intl_total_fees_actual,intl_total_fees_amortized,intl_total_fees_attrited
0,Advanced Diploma in Patisserie and Gastronomic...,TU,2019,C1,201901,2019-01-04,2019-11-28,6266.0,10360.0,6266.0,...,7916.0,10360.0,7916.0,10256.4,16626.0,16626.0,16522.4,18276.0,18276.0,18172.4
1,Advanced Diploma in Patisserie and Gastronomic...,TU,2019,C3,201908,2019-08-23,2019-11-28,6266.0,8288.0,6266.0,...,7916.0,8288.0,7916.0,8288.0,14554.0,14554.0,14554.0,16204.0,16204.0,16204.0
2,Advanced Diploma in Patisserie and Gastronomic...,TU,2020,C1,202001,2020-01-13,2020-11-29,6258.0,10360.0,6258.0,...,13708.0,10360.0,13708.0,10152.8,16618.0,16618.0,16410.8,24068.0,24068.0,23860.8
3,Advanced Diploma in Patisserie and Gastronomic...,TU,2020,C3,202008,2020-08-24,2020-11-29,6258.0,8288.0,6258.0,...,13708.0,8288.0,13708.0,8288.0,14546.0,14546.0,14546.0,21996.0,21996.0,21996.0
4,Advanced Diploma in Patisserie and Gastronomic...,TU,2021,C1,202101,2021-01-04,2021-11-28,6266.0,10360.0,6266.0,...,7916.0,10360.0,7916.0,10256.4,16626.0,16626.0,16522.4,18276.0,18276.0,18172.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955,TMSB_UNISA -Bachelor of Commerce (Accounting),TC,2019,C3,201908,2019-08-23,2022-02-20,4766.0,17724.0,4766.0,...,12216.0,17724.0,12157.2,17470.8,22490.0,22490.0,22178.0,29940.0,29940.0,29628.0
2956,TMSB_UNISA -Bachelor of Commerce (Accounting),TC,2020,C2,202003,2020-04-13,2020-11-29,5864.0,25320.0,5864.0,...,11164.0,25320.0,11017.4,24687.0,31184.0,31184.0,30404.4,36484.0,36484.0,35704.4
2957,TMSB_UNISA -Bachelor of Commerce (Accounting),TC,2020,C3,202008,2020-08-24,2020-11-29,2932.0,12660.0,2932.0,...,8232.0,12660.0,8232.0,12660.0,15592.0,15592.0,15592.0,20892.0,20892.0,20892.0
2958,TMSB_UNISA -Bachelor of Commerce (Accounting),TC,2021,C2,202103,2021-03-22,2021-11-21,6530.0,25320.0,6530.0,...,13980.0,25320.0,13833.0,24687.0,31850.0,31850.0,31070.0,39300.0,39300.0,38520.0


In [13]:
def calculate_first_year_fee():
    fin_fee = preprocess_finance_fees()
    
    # Select only the rows where the academic start date year is equal to the intake year, as we are calculating only the first year fees
    fee_by_cycle = fin_fee[(fin_fee['acad_start_date'].dt.year == fin_fee['intake_year'])].reset_index(drop=True)

    # Amortization formula is only applicable from 2023 onwards
    fee_by_cycle['amortized_nom'] = fee_by_cycle.apply(
        lambda row: row['amortized_nom'] if row['intake_year'] >= 2023 else row['amortized_denom'], axis=1
    )

    # Calculate first_year_fee by grouping the data by prog_name, campus, intake_year, intake_cycle, and intake
    first_year_fee = fee_by_cycle.groupby(['prog_name', 'campus', 'intake_year', 'intake_cycle', 'intake']).apply(
            lambda df: pd.Series({
                'fee_period_start': df['acad_start_date'].min(),
                'fee_period_end': df['acad_end_date'].max(),
                # Local Fees raw
                'loc_non_tuition_fees_actual': (df['loc_enrollment_fee'] + df['loc_resource_fee'] + 
                                                df['calsace_sci_fee_mult_loc'].fillna(0) * df['calsace_science_fee']).sum(),
                'loc_tuition_fees_actual': (df['loc_tuition_fee'] + df['loc_tuition_fee'] * 
                                                    df['calsace_fee_mult_loc'].fillna(0)).sum(),
                # Local fees adjusted for amortization
                'loc_non_tuition_fees_amortized': ((df['loc_enrollment_fee'] + df['loc_resource_fee'] + 
                                                    df['calsace_sci_fee_mult_loc'].fillna(0) * df['calsace_science_fee']) * 
                                                    df['amortized_nom'] / df['amortized_denom']).sum(),
                'loc_tuition_fees_amortized': ((df['loc_tuition_fee'] + df['loc_tuition_fee'] * 
                                        df['calsace_fee_mult_loc'].fillna(0)) * 
                                        df['amortized_nom'] / df['amortized_denom']).sum(),
                # Local fees adjusted for amortization and attrition
                'loc_non_tuition_fees_attrited': ((df['loc_enrollment_fee'] + df['loc_resource_fee'] + 
                                                    df['calsace_sci_fee_mult_loc'].fillna(0) * df['calsace_science_fee']) * 
                                                    df['amortized_nom'] / df['amortized_denom'] * (1-df['attrition'])).sum(),
                'loc_tuition_fees_attrited': ((df['loc_tuition_fee'] + df['loc_tuition_fee'] * 
                                        df['calsace_fee_mult_loc'].fillna(0)) * 
                                        df['amortized_nom'] / df['amortized_denom'] * (1-df['attrition'])).sum(),
                # International Fees raw
                'intl_non_tuition_fees_actual': (df['intl_enrollment_fee'] + df['loc_resource_fee'] + 
                                                df['intl_student_charges'] + df['intl_annual_fee'] + 
                                                df['calsace_sci_fee_mult_intl'].fillna(0) * df['calsace_science_fee']).sum(),
                'intl_tuition_fees_actual': (df['intl_tuition_fee'] + df['intl_tuition_fee'] * 
                                            df['calsace_fee_mult_intl'].fillna(0)).sum(),
                # International fees adjusted for amortization
                'intl_non_tuition_fees_amortized': ((df['intl_enrollment_fee'] + df['loc_resource_fee'] + 
                                            df['intl_student_charges'] + df['intl_annual_fee'] + 
                                            df['calsace_sci_fee_mult_intl'].fillna(0) * df['calsace_science_fee']) * 
                                            df['amortized_nom'] / df['amortized_denom']).sum(),
                'intl_tuition_fees_amortized': ((df['intl_tuition_fee'] + df['intl_tuition_fee'] * 
                                        df['calsace_fee_mult_intl'].fillna(0)) * 
                                        df['amortized_nom'] / df['amortized_denom']).sum(),
                # International fees adjusted for amortization and attrition
                'intl_non_tuition_fees_attrited': ((df['intl_enrollment_fee'] + df['loc_resource_fee'] + 
                                            df['intl_student_charges'] + df['intl_annual_fee'] + 
                                            df['calsace_sci_fee_mult_intl'].fillna(0) * df['calsace_science_fee']) * 
                                            df['amortized_nom'] / df['amortized_denom'] * (1-df['attrition'])).sum(),
                'intl_tuition_fees_attrited': ((df['intl_tuition_fee'] + df['intl_tuition_fee'] * 
                                        df['calsace_fee_mult_intl'].fillna(0)) * 
                                        df['amortized_nom'] / df['amortized_denom'] * (1-df['attrition'])).sum(),
            # amortization flags     
            'amortized_nom': df['amortized_nom'].sum(),
            'amortized_denom': df['amortized_denom'].sum(),
            })
        ).reset_index()
    
    return first_year_fee

In [14]:
first_year_fee = calculate_first_year_fee() 

Start preprocessing finance fee files...
Loading data...
Using manual finance fees data extraction...
Merging 'international total fees' data...
Merging 'academic calendar' data...
Merging 'CALSACE' data...
Calculating amortization related columns...
Preprocessing finance fee files completed.


In [17]:
def preprocess_first_year_fee():
    first_year_fee = calculate_first_year_fee()    
    
    # Unpivot first_year_fee to get long format
    first_year_fee_long = pd.melt(
        first_year_fee, 
        id_vars=['prog_name', 'campus', 'intake_year', 'intake_cycle', 'intake', 'fee_period_start', 'fee_period_end', 'amortized_nom', 'amortized_denom'],
        var_name='fee_type', 
        value_name='fee_amount'
    )

    # Create a new column 'market_segment' based on the fee_type column
    first_year_fee_long['market_segment'] = np.where(
        first_year_fee_long['fee_type'].str.contains('loc'), 'Domestic', 'International'
    )

    # Append 'Progression' market segment for Domestic rows
    progression_df = first_year_fee_long[first_year_fee_long['market_segment'] == 'Domestic'].copy()
    progression_df['market_segment'] = 'Progression'
    first_year_fee_long = pd.concat([first_year_fee_long, progression_df], ignore_index=True)

    # remove the 'loc_' and 'intl_' prefixes from the fee_type column
    first_year_fee_long['fee_type'] = first_year_fee_long['fee_type'].str.replace(r'loc_|intl_', '', regex=True)

    # Pivot the dataframe to get the final output
    first_year_fee_long = first_year_fee_long.pivot_table(
        index=['prog_name', 'campus', 'intake_year', 'intake_cycle', 'intake', 'fee_period_start', 'fee_period_end', 'market_segment', 'amortized_nom', 'amortized_denom'], 
        columns=['fee_type'], 
        values='fee_amount'
        ).reset_index()

    # Use vectorized comparison for is_amortized
    first_year_fee_long['is_amortized'] = first_year_fee_long['amortized_nom'].ne(first_year_fee_long['amortized_denom'])
    
    return first_year_fee_long

In [18]:
ff_long = preprocess_first_year_fee()

Start preprocessing finance fee files...
Loading data...
Using manual finance fees data extraction...
Merging 'international total fees' data...
Merging 'academic calendar' data...
Merging 'CALSACE' data...
Calculating amortization related columns...
Preprocessing finance fee files completed.


In [19]:
ff_long

fee_type,prog_name,campus,intake_year,intake_cycle,intake,fee_period_start,fee_period_end,market_segment,amortized_nom,amortized_denom,non_tuition_fees_actual,non_tuition_fees_amortized,non_tuition_fees_attrited,tuition_fees_actual,tuition_fees_amortized,tuition_fees_attrited,is_amortized
0,Advanced Diploma in Patisserie and Gastronomic...,TU,2019,C1,201901,2019-01-04,2019-11-28,Domestic,8.0,8.0,6266.0,6266.0,6266.0,10360.0,10360.0,10256.4,False
1,Advanced Diploma in Patisserie and Gastronomic...,TU,2019,C1,201901,2019-01-04,2019-11-28,International,8.0,8.0,7916.0,7916.0,7916.0,10360.0,10360.0,10256.4,False
2,Advanced Diploma in Patisserie and Gastronomic...,TU,2019,C1,201901,2019-01-04,2019-11-28,Progression,8.0,8.0,6266.0,6266.0,6266.0,10360.0,10360.0,10256.4,False
3,Advanced Diploma in Patisserie and Gastronomic...,TU,2019,C3,201908,2019-08-23,2019-11-28,Domestic,4.0,4.0,6266.0,6266.0,6266.0,8288.0,8288.0,8288.0,False
4,Advanced Diploma in Patisserie and Gastronomic...,TU,2019,C3,201908,2019-08-23,2019-11-28,International,4.0,4.0,7916.0,7916.0,7916.0,8288.0,8288.0,8288.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8875,TMSB_UNISA -Bachelor of Commerce (Accounting),TC,2021,C2,202103,2021-03-22,2021-11-21,International,8.0,8.0,13980.0,13980.0,13833.0,25320.0,25320.0,24687.0,False
8876,TMSB_UNISA -Bachelor of Commerce (Accounting),TC,2021,C2,202103,2021-03-22,2021-11-21,Progression,8.0,8.0,6530.0,6530.0,6383.0,25320.0,25320.0,24687.0,False
8877,TMSB_UNISA -Bachelor of Commerce (Accounting),TC,2021,C3,202108,2021-08-23,2021-12-19,Domestic,5.0,5.0,3590.0,3590.0,3590.0,12660.0,12660.0,12660.0,False
8878,TMSB_UNISA -Bachelor of Commerce (Accounting),TC,2021,C3,202108,2021-08-23,2021-12-19,International,5.0,5.0,11040.0,11040.0,11040.0,12660.0,12660.0,12660.0,False
